# Sanity check for Colmap points versus the satellite image

As a sanity check, what happens if you create a snapshot (birdseye view) of the collmap point cloud, and then measure distances between relevant points on the field.
- You can make the measurements in yards if the field is 100 yd long

How do the distances in the collmap point cloud image compare to those in the satellite photo

Are the distances comparable?  

Wondering if the collmap dimensions are a bit skewed, or even if the painting on the field might have been updated between your photos and the satellite photo

If the dimensions between key features measured in the collmap don’t match the satellite image, then that gives us a good clue about where to start next.

In [1]:
import numpy as np
import cv2
import open3d as o3d
import plotly.graph_objects as go
import plotly.io as pio
from scipy.spatial.transform import Rotation as R
from scipy.spatial import cKDTree
import imageio
# %matplotlib qt
import matplotlib.pyplot as plt
from matplotlib.path import Path

from groundNAV_agentEDITING import *
from colmapParsingUtils import *

# SAVE YOUR WORK
%load_ext autoreload
%autoreload 2
%autosave 180

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Autosaving every 180 seconds


In [2]:
# Load in necessary parameters for gNAV agent 
# Define Class Parameters 

images_colm  = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/images.txt"
cameras_colm = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/cameras.txt"
pts3d_colm = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/points3D_f.txt"

# Load in satellite reference image
sat_ref = "TTurf/TurfSat.jpg"

# Images selected for local corrections
image_1 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9446.JPEG" #ID:4

# Organize for agent params
images = [image_1]

In [3]:
# Create class
gnav = gNAV_agent(images_colm, cameras_colm, pts3d_colm, images, sat_ref)

# Grab raw points and RGB data for scene and reference cloud
scene_pts, rgb_data = gnav.grab_pts(gnav.pts3d_c)
ref_pts, ref_rgb = gnav.ref_pts, gnav.ref_rgb

In [4]:
# Use ground plane pts to set reference frame 
# Need gravity and height
pts_gnd_idx = np.array([25440, 25450, 25441, 25449, 25442, 25445, 103922, 103921, 103919, 103920])
tform_ref_frame = gnav.set_ref_frame(pts_gnd_idx)
tform_ref_frame_pts = gnav.inv_homog_transform(tform_ref_frame)
# print("\nReference frame transformation\n", tform_ref_frame)

In [5]:
# Transform all points to the new coordinate system 
# Not necessary since we aren't using the cloud, but a good visual check for coord frame
tform_ref_frame_inv = gnav.inv_homog_transform(tform_ref_frame)
origin_ref, scene_pts_ref, scene_vec_ref = gnav.unit_vec_tform(scene_pts, gnav.origin_w, tform_ref_frame_inv)
# print(origin_ref)
# Transform scene cloud to 2D (also as a visual check)
# Note: 2d projection will look off with z=-1; see TTurf_v2 for cropping method
scene_ref_2d = gnav.proj_2d_scene(scene_pts_ref)
# print(scene_ref_2d)

In [6]:
# CROP UNNECESSARY colmap cloud points for birds-eye view 
# Crop points outside of boundary 
print(max(scene_pts_ref[:,2]))
x_min = -20
x_max = 20
y_min = -15
y_max = 15
z_min = 0
z_max = 2

# boundary_pts_idx = np.argwhere((pts_2D[:,0] < x_min) | (pts_2D[:,0] > x_max) | (pts_2D[:,1] < y_min) | (pts_2D[:,1] > y_max))
new_pts_idx = np.argwhere((scene_pts_ref[:,0] > x_min) & (scene_pts_ref[:,0] < x_max) & (scene_pts_ref[:,1] > y_min) &
                          (scene_pts_ref[:,1] < y_max))# & (scene_pts_ref[:,2] > z_min) & (scene_pts_ref[:,2] < z_max))
# Get new points and rgb data 
new_pts = scene_pts_ref[new_pts_idx[:,0],:]
rgb_data_new = rgb_data[new_pts_idx[:,0],:]
# Get 2d projection of points
scene_ref_2d = gnav.proj_2d_scene(new_pts)
print(new_pts)

44.175401880708456
[[11.74604814 -6.40375344 39.53529655]
 [12.13326906 -6.77923053 38.09590283]
 [11.49564909 -4.10254722 37.60552444]
 ...
 [ 0.58305242 -1.86513577 40.00641745]
 [ 0.08944373 -2.65965277 40.00618831]
 [-0.34202601 -3.33846128 40.00245484]]


In [23]:
# CHECKING CROPPED CLOUD 

# Use open3d to create point cloud visualization 
# Create visualization 
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Originial scene with ORIGINAL ORIGIN")

# Create axes @ origin
axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# Create point cloud for scene points
scene_cloud = o3d.geometry.PointCloud()
scene_cloud.points = o3d.utility.Vector3dVector(scene_pts)
scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data)

# # Create point cloud for satellite reference points
# ref_cloud = o3d.geometry.PointCloud()
# ref_cloud.points = o3d.utility.Vector3dVector(ref_pts)
# ref_cloud.colors = o3d.utility.Vector3dVector(ref_rgb)

# Add necessary geometries to visualization 
vis.add_geometry(axis_origin)
vis.add_geometry(scene_cloud)
# vis.add_geometry(ref_cloud)

# # Size options (jupyter gives issues when running this multiple times, but it looks better)
# render_option = vis.get_render_option()
# render_option.point_size = 2

# Run and destroy visualization 
vis.run()
vis.destroy_window()

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 74
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 74
pci id for fd 74: 8086:a7a0, driver iris
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using DRI3 for screen 0


In [24]:
# Original guess to align clouds

# Implementing an initial guess for the local image 

# SCALE for initial guess 
scale = 2700 # Decent average guess (not actually, need to change)

# Translation and Rotation for initial guess
x = 395
y = 317
yaw = -1

tform_guess = gnav.tform_create(x,y,0,0,0,yaw)

bestguess_2d = scene_ref_2d.copy()
bestguess_2d[:,:2] *= scale
# Get new points 
__, bestguess_2d_pts, bestguess_2d_vec = gnav.unit_vec_tform(bestguess_2d, gnav.origin_w, tform_guess)

In [25]:
# PLOTTING THE ORIGINAL SCENE WITH THE ORIGINAL ORIGIN 

# Use open3d to create point cloud visualization 
# Create visualization 
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Originial scene with ORIGINAL ORIGIN")

# Create axes @ origin
axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100)

# Create point cloud for scene points
scene_cloud = o3d.geometry.PointCloud()
scene_cloud.points = o3d.utility.Vector3dVector(bestguess_2d_pts)
scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data_new)

# Create point cloud for satellite reference points
ref_cloud = o3d.geometry.PointCloud()
ref_cloud.points = o3d.utility.Vector3dVector(ref_pts)
ref_cloud.colors = o3d.utility.Vector3dVector(ref_rgb)

# Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
vis.add_geometry(scene_cloud)
vis.add_geometry(ref_cloud)

# # Size options (jupyter gives issues when running this multiple times, but it looks better)
# render_option = vis.get_render_option()
# render_option.point_size = 2

# Run and destroy visualization 
vis.run()
vis.destroy_window()

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 74
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 74
pci id for fd 74: 8086:a7a0, driver iris
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using DRI3 for screen 0
